<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/shakespeare_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference of a tiny Shakespeare LLM

In [3]:
from datetime import datetime
print("Current Date and Time:", datetime.now())

Current Date and Time: 2024-11-28 12:45:17.046424


In [4]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
gpt_neox_colabDirName = "GPT-NeoX-Colab"
gpt_neox_colabDir = f"{workspaceDir}/{gpt_neox_colabDirName}"

In [2]:
#!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.5.5 x86_64-unknown-linux-gnu
no checksums to verify
installing to /root/.local/bin
  uv
  uvx
everything's installed!

To add $HOME/.local/bin to your PATH, either restart your shell or run:

    source $HOME/.local/bin/env (sh, bash, zsh)
    source $HOME/.local/bin/env.fish (fish)


# Cloning Git Repos

In [45]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
#%cd {gpt_neox_colabDir}
#!uv sync -q --dev
#!uv run pip install -q -e .

/content
fatal: destination path 'GPT-NeoX-Colab' already exists and is not an empty directory.
/content/GPT-NeoX-Colab
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for gtp_neox_colab (pyproject.toml) ... done
CPU times: user 242 ms, sys: 35.3 ms, total: 278 ms
Wall time: 33.9 s


In [54]:
# This runs python in the virtual environment
#%%bash -s "$gpt_neox_colabDir"
#source "$1/.venv/bin/activate"
#python -c "
#from dotenv import load_dotenv
#import os
#load_dotenv('$1/.env')
#import gpt_neox_colab
#gpt_neox_colab.utils.colab.fetch_data('data/shakespeare/model.tar.gz')
#"

Data retrieval successful.


In [60]:
%pip install -q dvc[s3]
%cd {gpt_neox_colabDir}
!dvc pull {'data/shakespeare/model.tar.gz'}
%cd {gpt_neox_colabDir}/data/shakespeare
!tar -xzf model.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.7 MB/s eta 0:00:00
/content/GPT-NeoX-Colab
Fetching
!
  0% |          |0/? [00:00<?,    ?files/s]
Fetching
Building workspace index          |3.00 [00:00,  968entry/s]
Comparing indexes          |4.00 [00:00,  993entry/s]
Applying changes          |0.00 [00:00,     ?file/s]
Everything is up to date.


# Inference with Hugging Face

In [4]:
from transformers import GPTNeoXForCausalLM
import torch

# Move to model directory
%cd {gpt_neox_colabDir}

# Assuming CharLevelTokenizer is properly imported and instantiated
from gpt_neox_colab import CharLevelTokenizer
tokenizer = CharLevelTokenizer.CharLevelTokenizer(vocab_size=512)

# Load your model
model_path = "data/shakespeare"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Define a simple char-level tokenizer if not provided
def char_level_tokenize(text):
    return tokenizer.tokenize(text)

def char_level_detokenize(tokens):
    return tokenizer.detokenize(tokens)

# Set the model to evaluation mode
model.eval()


/content/GPT-NeoX-Colab


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(512, 256)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-3): 4 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=256, out_features=768, bias=True)
          (dense): Linear(in_features=256, out_features=256, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=256, out_features=1024, bias=True)
          (dense_4h_to_h): Linear(in_features=1024, out_features=256, bias=True)
          (act)

In [6]:
# Prompt the user for input
input_text = input("Enter your prompt: ")

# Tokenize and prepare input
input_ids = torch.tensor([tokenizer.tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode and print the generated text
generated_text = tokenizer.detokenize(output[0].tolist())
print("Generated text:", generated_text)

Enter your prompt: Mark is a 
Generated text: Mark is a begger. But to the counted looks. Than thou and my cous. Thereloved so office, And as you be the sea; whose that CLIFFORD: Of my should duke to speak. And more shall deed. And may man? while
